<a href="https://colab.research.google.com/github/iGhostlp/Proyecto_MELI/blob/master/Proyecto_MELI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prueba usando base de datos kaggle**

Librerias

In [ ]:
from pyspark import *
import pandas as pd
import numpy as np
from pyspark.sql import SQLContext
import pyspark.sql.functions as sqlf
import yaml
# import validations
#import matplotlib.pyplot as plt
from pyspark.sql.types import *
import pyarrow.parquet as pq
#Master
#from pyspark.sql import functions as F
#from pyspark.sql import SparkSession
pd.options.display.max_columns = 999
pd.set_option('max_colwidth',200)
#spark.conf.set("spark.sql.execution.arrow.enabled","false")
import kagglehub
from kagglehub import KaggleDatasetAdapter
import os

In [ ]:
# Descarga la última versión del dataset
dataset_path = kagglehub.dataset_download("davidcariboo/player-scores")

# Obtener todos los archivos CSV en el dataset
csv_files = [file for file in os.listdir(dataset_path) if file.endswith(".csv")]
print("Archivos encontrados:", csv_files)

# Diccionario para almacenar los DataFrames
df_dict = {}

# Leer y almacenar cada CSV en el diccionario
for file in csv_files:
    df_name = file.replace(".csv", "")  # Usar el nombre del archivo como clave
    df_dict[df_name] = pd.read_csv(os.path.join(dataset_path, file))

# Ver el nombre de las claves del diccionario y el tamaño de cada DataFrame
for name, df in df_dict.items():
    print(f"DataFrame para {name}:")
    print(df.head(), "\n")  # Muestra las primeras filas de cada DataFrame

In [143]:
df_appearances = df_dict["appearances"]
df_club_games = df_dict["club_games"]
df_club_games = df_dict["club_games"]
df_competitions = df_dict["competitions"]
df_game_events = df_dict["game_events"]
df_games = df_dict["games"]
df_players = df_dict["players"]

In [ ]:
df_competitions.head()

In [169]:
df1 = df_competitions.filter(items=['competition_id','name','country_name','domestic_league_code','confederation'])

In [170]:
print(df1.head(10))

  competition_id                           name country_name  \
0            CIT                      italy-cup        Italy   
1           NLSC           johan-cruijff-schaal  Netherlands   
2            GRP                kypello-elladas       Greece   
3           POSU  supertaca-candido-de-oliveira     Portugal   
4           RUSS              russian-super-cup       Russia   
5            SUC                      supercopa        Spain   
6            USC                 uefa-super-cup          NaN   
7            DK1                    superligaen      Denmark   
8             EL                  europa-league          NaN   
9            ES1                         laliga        Spain   

  domestic_league_code confederation  
0                  IT1        europa  
1                  NL1        europa  
2                  GR1        europa  
3                  PO1        europa  
4                  RU1        europa  
5                  ES1        europa  
6                  NaN

In [136]:
# Paso 1: Selección de liga (Solo ligas principales de Europa)
ligas_europa = ["Premier League", "La Liga", "Bundesliga", "Serie A", "Ligue 1"]

print("\nElige una liga de Europa:")
for i, liga in enumerate(ligas_europa, 1):
    print(f"{i}. {liga}")

opcion = int(input("Ingresa el número de la liga: ").strip())
liga_elegida = ligas_europa[opcion - 1] if 1 <= opcion <= len(ligas_europa) else exit()

# Filtrar el dataset por la liga elegida
df_liga = df_final[df_final["competition_id"] == liga_elegida]

# Agrupar por equipo y sumar los goles
goles_por_equipo = df_liga.groupby("home_club_name")["goals"].sum()

# Encontrar el equipo con más goles
equipo_sugerido = goles_por_equipo.idxmax() if not goles_por_equipo.empty else "No encontrado"

# Mostrar resultado
print(f"\n⚽ ¡El equipo con más goles en {liga_elegida} es {equipo_sugerido}! ⚽")


Elige una liga de Europa:
1. Premier League
2. La Liga
3. Bundesliga
4. Serie A
5. Ligue 1
Ingresa el número de la liga: 1

⚽ ¡El equipo con más goles en Premier League es No encontrado! ⚽


In [124]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("davidcariboo/player-scores")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/davidcariboo/player-scores/versions/595


In [ ]:
import pandas as pd

# Cargar el dataset de Kaggle "player-scores"
df = pd.read_csv("player_scores.csv")

# Paso 1: Selección de liga (Solo ligas principales de Europa)
ligas_europa = ["Premier League", "La Liga", "Bundesliga", "Serie A", "Ligue 1"]

print("\nElige una liga de Europa:")
for i, liga in enumerate(ligas_europa, 1):
    print(f"{i}. {liga}")

opcion = int(input("Ingresa el número de la liga: ").strip())
liga_elegida = ligas_europa[opcion - 1] if 1 <= opcion <= len(ligas_europa) else exit()

# Filtrar el dataset por la liga elegida
df_liga = df[df["league"] == liga_elegida]

# Agrupar por equipo y sumar los goles
goles_por_equipo = df_liga.groupby("team")["goals"].sum()

# Encontrar el equipo con más goles
equipo_sugerido = goles_por_equipo.idxmax() if not goles_por_equipo.empty else "No encontrado"

# Mostrar resultado
print(f"\n⚽ ¡El equipo con más goles en {liga_elegida} es {equipo_sugerido}! ⚽")

# **Nuevo metodo YT**

In [ ]:
pip install LanusStats

In [3]:
pip install tyrone_mings

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1253 sha256=8abc7b822a3d9ea921b8490ef52395330af14129bf5a172ac62c036ed91ffc16
  Stored in directory: /root/.cache/pip/wheels/d4/c8/5b/b5be9c20e5e4503d04a6eac8a3cd5c2393505c29f02bea0960
  Created wheel for slimit: filename=slimit-0.8.1-py3-none-any.whl size=79435 sha256=836e0e65a42416403c70ffbfb1f5806ad18ddd118e99fae412b6024b4ba56dfd
  Stored in directory: /root/.cache/pip/wheels/31/31/61/62edc1218576ea1737878bb64623e52117c45637fc54118c04
Successfully built bs4 slimit
  Attempting uninstall: bs4
    Found existing installation: bs4 0.0.2
    Uninstalling bs4-0.0.2:
      Successfully uninstalled bs4-0.0.2


In [5]:
pip install devtools

In [7]:
install.packages("worldfootballR")

NameError: name 'install' is not defined

Inicializacion Librerias

In [114]:
from pyspark import *
import pandas as pd
import numpy as np
from pyspark.sql import SQLContext
import pyspark.sql.functions as sqlf
import yaml
# import validations
#import matplotlib.pyplot as plt
from pyspark.sql.types import *
import pyarrow.parquet as pq
#Master
#from pyspark.sql import functions as F
#from pyspark.sql import SparkSession
#import pandas as pd
pd.options.display.max_columns = 999
pd.set_option('max_colwidth',200)
#spark.conf.set("spark.sql.execution.arrow.enabled","false")
import LanusStats as ls
import tyrone_mings as tm


Creo Listas para diferencias ligas UEFA de CONMEBOL

In [24]:
ls.get_possible_leagues

<function LanusStats.functions.get_possible_leagues(league, season, page)>

In [ ]:
df = tm_pull
output_dict = tm_pull(player_page, player_bio = True, output = 'pandas')

In [ ]:
consola1 = str(input("Que decada buscas?: "))

Que decada buscas?: 90


In [ ]:
Parametro_decada = consola1

In [ ]:
club_urls = get_club_urls_from_league_page("https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1")

IndexError: list index out of range

In [59]:
from pyspark import *
import numpy as np
from pyspark.sql import SQLContext
import pyspark.sql.functions as sqlf
import yaml
# import validations
#import matplotlib.pyplot as plt
from pyspark.sql.types import *
import pyarrow.parquet as pq
pd.options.display.max_columns = 999
pd.set_option('max_colwidth',200)
#spark.conf.set("spark.sql.execution.arrow.enabled","false")
import LanusStats as ls
import tyrone_mings as tm
import kagglehub
import pandas as pd
import requests



In [48]:
def obtener_equipo_top_goles(liga):
    """Filtra el dataset por liga y calcula qué equipo hizo más goles."""
    df_liga = df[df["competition_code"] == liga]  # Filtrar por liga
    top_equipo = df_liga.groupby("team")["goals"].sum().idxmax()  # Sumar goles y encontrar el máximo
    return top_equipo

In [ ]:
def obtener_enlace_mercadolibre(equipo):
    """Genera un enlace de búsqueda en MercadoLibre con el nombre del equipo."""
    base_url = "https://www.mercadolibre.com.ar/ar/search"
    params = {"q": f"camiseta {equipo_sugerido}", "condition": "new"}
    return f"{base_url}?q=camiseta+{equipo.replace(' ', '+')}"

# 📌 Paso 1: Preguntar la región
region = input("¿Quieres un equipo de Europa o Sudamérica? ").strip().lower()

ligas_europa = ["Premier League", "La Liga", "Bundesliga", "Serie A", "Ligue 1"]
ligas_sudamerica = ["Liga Argentina", "Brasileirão", "Liga Chilena", "Liga Uruguaya"]

ligas = ligas_europa if region == "europa" else ligas_sudamerica

# 📌 Paso 2: Preguntar la liga
print("\nElige una liga:")
for i, liga in enumerate(ligas, 1):
    print(f"{i}. {liga}")

opcion = int(input("Ingresa el número de la liga: ").strip())
liga_elegida = ligas[opcion - 1] if 1 <= opcion <= len(ligas) else exit()

# 📌 Paso 3: Obtener equipo con más goles
equipo_sugerido = obtener_equipo_top_goles(liga_elegida)

# 📌 Mostrar resultado
print(f"\n⚽ ¡El equipo con más goles en {liga_elegida} es {equipo_sugerido}! ⚽")


# 📌 Paso 5: Generar enlace de compra en MercadoLibre
enlace_compra = obtener_enlace_mercadolibre(equipo_sugerido)

# 📌 Mostrar resultado
print(f"\n🎉 ¡Equipo sugerido: {equipo_sugerido}! 🎉")
print(f"🔗 Aca te dejo una busqueda para comprar tu camiseta: {enlace_compra}")

In [99]:
def obtener_competicion_id(nombre_liga):
    """Busca el ID de la competición según el nombre de la liga."""
    url = f"https://transfermarkt-api.fly.dev/competitions/search/{nombre_liga}"
    response = requests.get(url)
    if response.status_code == 200:
        competiciones = response.json()
        if competiciones:
            return competiciones[0]["competition_id"]  # Tomamos el primer resultado
    return None


In [112]:
def obtener_competicion_id(nombre_liga):
    """Busca el ID de la competición según el nombre de la liga."""
    # Modificación: URL de la API actualizada para que coincida con el formato esperado
    url = f"https://transfermarkt-api.fly.dev/competitions/search/Premier%20League?page_number=1"
    response = requests.get(url)
    if response.status_code == 200:
        competiciones = response.json()
        if competiciones:
            return competiciones[0]["name"]  # Tomamos el primer resultado
            print(competiciones)
    return None
